# Amazon SageMaker Processing Job 


기계 학습 (ML) 프로세스는 몇 단계로 구성됩니다. 먼저, 다양한 ETL 작업으로 데이터를 수집 한 다음 data의 pre-processing, 전통적인 기법 또는 사전 knowledge를 이용하여 데이터의 feature화, 마지막으로 알고리즘을 이용한 ML 모델을 학습합니다.

Apache Spark와 같은 분산 데이터 처리 프레임 워크는 학습을 위해 dataset의 pre-processing하는데 사용합니다. 이 노트북에서는 Amazon SageMaker Processing에서 기본 설치된 Apache Spark의 기능을 활용하여 처리 워크로드를 실행합니다.

![](img/prepare_dataset_bert.png)

![](img/processing.jpg)


# Setup Environment


* 모델 학습에 사용되는 S3 bucket과 prefix 가 필요합니다.
* 학습과 processing을 위해 IAM role은 dataset에 액세스가 가능해야 합니다.

In [1]:
import sagemaker
from time import gmtime, strftime
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Setup Input Data

In [2]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-east-1-322537213286/amazon-reviews-pds/tsv/


In [3]:
!aws s3 ls $s3_input_data

2020-07-31 08:48:04   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-07-31 08:48:07   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz
2020-07-31 08:48:08  193389086 amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz


# Processing Job을 수행할 Spark Docker Image

이 HOL에서는 `./container` 폴더 내에 Spark container 이미지를 포함합니다. container는 모든 Spark 구성의 부트스트랩을 처리하고 `spark-submit` CLI를 wrapper해서 제공합니다. 상위 레벨에서는,

* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application

container 빌드와 push 절차가 완료된 후 dataset의 처리를 수행하는 관리형 분산 Spark 어플리케이션을 수행사는 것은 Amazon SageMaker Python SDK 사용합니다.

In [4]:
docker_repo = 'amazon-reviews-spark-processor'
docker_tag = 'latest'

In [5]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  4.385MB
Step 1/37 : FROM openjdk:8-jre-slim
 ---> fec2d7a65b07
Step 2/37 : RUN apt-get update
 ---> Using cache
 ---> e88bd9f80f9f
Step 3/37 : RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
 ---> Using cache
 ---> b898d99fac21
Step 4/37 : RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
 ---> Using cache
 ---> 0e16d95264d8
Step 5/37 : RUN apt-get clean
 ---> Using cache
 ---> 4f3243262295
Step 6/37 : RUN rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> d9634587550e
Step 7/37 : ENV PYTHONHASHSEED 0
 ---> Using cache
 ---> 49b8b187803d
Step 8/37 : ENV PYTHONIOENCODING UTF-8
 ---> Using cache
 ---> 08c7b9e2cf28
Step 9/37 : ENV PIP_DISABLE_PIP_VERSION_CHECK 1
 ---> Using cache
 ---> af67654cc520
Step 10/37 : ENV HADOOP_VERSION 3.2.1
 ---> Using cache
 ---> 62ce8d352867
Step 11/37 : ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
 ---> Using cache
 ---> 854a7d7d0d70
Step 12/37 : ENV HADOOP

Spark container의 Amazon Elastic Container Registry(Amazon ECR) 리포지토리를 생성하고 image를 push합니다.

In [6]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

322537213286.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor:latest


### ECR repository 생성과 docker image를 push하기

In [7]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


### `RepositoryNotFoundException` 오류는 무시하셔도 됩니다. 즉시 repository를 생성합니다.

In [8]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:322537213286:repository/amazon-reviews-spark-processor",
            "registryId": "322537213286",
            "repositoryName": "amazon-reviews-spark-processor",
            "repositoryUri": "322537213286.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor",
            "createdAt": 1596262176.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            }
        }
    ]
}


In [9]:
!docker tag $docker_repo:$docker_tag $image_uri

In [10]:
!docker push $image_uri

The push refers to repository [322537213286.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor]

17e4a6cc: Preparing 
70cf88ab: Preparing 
9c81145b: Preparing 
1c38d96f: Preparing 
1a6011de: Preparing 
60e645c3: Preparing 
7573735c: Preparing 
1887bc1e: Preparing 
158a1221: Preparing 
582e15ed: Preparing 
47ad5106: Preparing 
a7f65aee: Preparing 
0eeb3a13: Preparing 
b9f2d7c4: Preparing 
a4d413ef: Preparing 
0e645c3: Waiting g 
38d128fe: Preparing 
e510e849: Preparing 
25a32043: Layer already exists latest: digest: sha256:00c3463d03d4d50e1c03f0e4d15b935ee0117645e7da3fc889256dffea6cd42e size: 4318


# Amazon SageMaker Processing Jobs 으로 Job 수행

Amazon SageMaker Python SDK를 사용하여 Processing job을 실행합니다. Spark container와 job configuration에서 processing에 대한 Spark ML script를 사용합니다.

In [11]:
!pygmentize src_dir/preprocess-spark-text-to-bert.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv
import collections
import subprocess
import sys
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pip', '--upgrade'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'wrapt', '--upgrade', '--ignore-installed'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0', '--ignore-installed'])
import tensorflow as tf
print(tf.__version__)
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
from transformers import DistilBertTokenizer

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import split
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *

tokenizer = DistilBertTokenizer.from_pretrained('distil

        StructField('verified_purchase', StringType(), True),
        StructField('review_headline', StringType(), True),
        StructField('review_body', StringType(), True),
        StructField('review_date', StringType(), True)
    ])
    
    df_csv = spark.read.csv(path=s3_input_data,
                            sep='\t',
                            schema=schema,
                            header=True,
                            quote=None)
    df_csv.show()

    # This dataset should already be clean, but always good to double-check
    print('Showing null review_body rows...')
    df_csv.where(col('review_body').isNull()).show()

    print('Showing cleaned csv')
    df_csv_dropped = df_csv.na.drop(subset=['review_body'])
    df_csv_dropped.show()

    # TODO:  Balance
    
    features_df = df_csv_dropped.select(['star_rating', 'review_body'])
    features_df.show()

    tfrecord_schema = StructType([
      StructField("input_ids", ArrayType(IntegerType(), False)),
      St

In [12]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-processor',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=2, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.xlarge',
                            env={'mode': 'python'})

# Setup Output Data

In [13]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-processor-{}'.format(timestamp_prefix)

In [14]:
train_data_bert_output = 's3://{}/{}/output/bert-train'.format(bucket, output_prefix)
validation_data_bert_output = 's3://{}/{}/output/bert-validation'.format(bucket, output_prefix)
test_data_bert_output = 's3://{}/{}/output/bert-test'.format(bucket, output_prefix)

print(train_data_bert_output)
print(validation_data_bert_output)
print(test_data_bert_output)

s3://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-2020-08-01-06-14-52/output/bert-train
s3://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-2020-08-01-06-14-52/output/bert-validation
s3://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-2020-08-01-06-14-52/output/bert-test


In [15]:
from sagemaker.processing import ProcessingOutput

processor.run(code='./src_dir/preprocess-spark-text-to-bert.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_train_data', train_data_bert_output,
                         's3_output_validation_data', validation_data_bert_output,
                         's3_output_test_data', test_data_bert_output,                         
              ],
              # We need this dummy output to allow us to call 
              #    ProcessingJob.from_processing_name() later 
              #    to describe the job and poll for Completed status
              outputs=[
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='dummy-output',
                                        source='/opt/ml/processing/output')
              ],          
              logs=True,
              wait=False
)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  spark-amazon-reviews-processor-2020-08-01-06-14-52-123
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-322537213286/spark-amazon-reviews-processor-2020-08-01-06-14-52-123/input/code/preprocess-spark-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-322537213286/spark-amazon-reviews-processor-2020-08-01-06-14-52-123/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]


In [16]:
from IPython.core.display import display, HTML

spark_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, spark_processing_job_name)))


In [17]:
from IPython.core.display import display, HTML

# This is different than the job name because we are not using ProcessingOutput's in this Spark ML case.
spark_processing_job_s3_output_prefix = output_prefix

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, spark_processing_job_s3_output_prefix, region)))


# List Processing Jobs through boto3 Python SDK

In [18]:
import boto3

client = boto3.client('sagemaker')
client.list_processing_jobs()

{'ProcessingJobSummaries': [{'ProcessingJobName': 'spark-amazon-reviews-processor-2020-08-01-06-14-52-123',
   'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:322537213286:processing-job/spark-amazon-reviews-processor-2020-08-01-06-14-52-123',
   'CreationTime': datetime.datetime(2020, 8, 1, 6, 14, 52, 478000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 8, 1, 6, 14, 52, 478000, tzinfo=tzlocal()),
   'ProcessingJobStatus': 'InProgress'},
  {'ProcessingJobName': 'sagemaker-scikit-learn-2020-08-01-05-52-12-044',
   'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:322537213286:processing-job/sagemaker-scikit-learn-2020-08-01-05-52-12-044',
   'CreationTime': datetime.datetime(2020, 8, 1, 5, 52, 12, 520000, tzinfo=tzlocal()),
   'ProcessingEndTime': datetime.datetime(2020, 8, 1, 5, 56, 59, 919000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2020, 8, 1, 5, 56, 59, 922000, tzinfo=tzlocal()),
   'ProcessingJobStatus': 'Completed'},
  {'ProcessingJobName':

# Please Wait Until the Processing Job Completes
Re-run this next cell until the job status shows `Completed`.

In [19]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=spark_processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-322537213286/spark-amazon-reviews-processor-2020-08-01-06-14-52-123/input/code/preprocess-spark-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-322537213286/spark-amazon-reviews-processor-2020-08-01-06-14-52-123/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]}, 'ProcessingJobName': 'spark-amazon-reviews-processor-2020-08-01-06-14-52-123', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 2, 'InstanceType': 'ml.r5.xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '322537213286.dkr.ecr.us-east-1.amazonaws.c

In [ ]:
running_processor.wait()

2020-08-01 06:18:38,961 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = algo-1/10.2.109.111
STARTUP_MSG:   args = [-format, -force]
STARTUP_MSG:   version = 3.2.1
STARTUP_MSG:   classpath = /usr/hadoop-3.2.1/etc/hadoop:/usr/hadoop-3.2.1/share/hadoop/common/lib/kerb-crypto-1.0.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jackson-databind-2.9.8.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jackson-mapper-asl-1.9.13.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/woodstox-core-5.0.3.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jetty-webapp-9.3.24.v20180605.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/token-provider-1.0.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/kerb-server-1.0.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/kerby-pkix-1.0.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jcip-annotations-1.0-1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jetty-io-9.3.24.v2018060

Starting nodemanagers
localhost: /usr/hadoop-3.2.1/bin/../libexec/hadoop-functions.sh: line 982: ssh: command not found
2020-08-01 06:18:52,428 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-08-01 06:18:53.340334: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-08-01 06:18:53.340420: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-08-01 06:18:53.340429: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2.1.0
#

2020-08-01 06:18:59,691 INFO impl.YarnClientImpl: Submitted application application_1596262730870_0001
2020-08-01 06:18:59,694 INFO cluster.SchedulerExtensionServices: Starting Yarn extension services with app application_1596262730870_0001 and attemptId None
2020-08-01 06:19:00,704 INFO yarn.Client: Application report for application_1596262730870_0001 (state: ACCEPTED)
2020-08-01 06:19:00,707 INFO yarn.Client: 
#011 client token: N/A
#011 diagnostics: AM container is launched, waiting for AM container to Register with RM
#011 ApplicationMaster host: N/A
#011 ApplicationMaster RPC port: -1
#011 queue: default
#011 start time: 1596262739576
#011 final status: UNDEFINED
#011 tracking URL: http://algo-1:8088/proxy/application_1596262730870_0001/
#011 user: root
2020-08-01 06:19:01,709 INFO yarn.Client: Application report for application_1596262730870_0001 (state: ACCEPTED)
2020-08-01 06:19:02,711 INFO yarn.Client: Application report for application_1596262730870_0001 (state: ACCEPTED)
20

2020-08-01 06:19:31,312 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 2622 ms on algo-2 (executor 1) (1/1)
2020-08-01 06:19:31,314 INFO cluster.YarnScheduler: Removed TaskSet 0.0, whose tasks have all completed, from pool 
2020-08-01 06:19:31,319 INFO scheduler.DAGScheduler: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0) finished in 2.721 s
2020-08-01 06:19:31,322 INFO scheduler.DAGScheduler: Job 0 finished: showString at NativeMethodAccessorImpl.java:0, took 2.752107 s
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+------------

2020-08-01 06:42:28,361 INFO scheduler.TaskSetManager: Finished task 1.0 in stage 6.0 (TID 7) in 1368079 ms on algo-2 (executor 1) (1/2)
2020-08-01 07:34:41,171 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 6.0 (TID 6) in 4500890 ms on algo-2 (executor 1) (2/2)
2020-08-01 07:34:41,171 INFO cluster.YarnScheduler: Removed TaskSet 6.0, whose tasks have all completed, from pool 
2020-08-01 07:34:41,173 INFO scheduler.DAGScheduler: ResultStage 6 (save at NativeMethodAccessorImpl.java:0) finished in 4500.933 s
2020-08-01 07:34:41,173 INFO scheduler.DAGScheduler: Job 6 finished: save at NativeMethodAccessorImpl.java:0, took 4500.935914 s
2020-08-01 07:34:41,905 INFO datasources.FileFormatWriter: Write Job c88f19af-c3b4-409c-b311-a7612bc49853 committed.
2020-08-01 07:34:41,909 INFO datasources.FileFormatWriter: Finished processing stats for write job c88f19af-c3b4-409c-b311-a7612bc49853.
Wrote to output file:  s3a://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-20

2020-08-01 07:57:27,275 INFO scheduler.TaskSetManager: Finished task 1.0 in stage 7.0 (TID 9) in 1364776 ms on algo-2 (executor 1) (1/2)
2020-08-01 08:49:14,736 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 7.0 (TID 8) in 4472239 ms on algo-2 (executor 1) (2/2)
2020-08-01 08:49:14,736 INFO cluster.YarnScheduler: Removed TaskSet 7.0, whose tasks have all completed, from pool 
2020-08-01 08:49:14,737 INFO scheduler.DAGScheduler: ResultStage 7 (save at NativeMethodAccessorImpl.java:0) finished in 4472.274 s
2020-08-01 08:49:14,738 INFO scheduler.DAGScheduler: Job 7 finished: save at NativeMethodAccessorImpl.java:0, took 4472.277808 s
2020-08-01 08:49:15,286 INFO datasources.FileFormatWriter: Write Job a8f8cec1-2211-43a4-ab0b-34dab34d62ef committed.
2020-08-01 08:49:15,286 INFO datasources.FileFormatWriter: Finished processing stats for write job a8f8cec1-2211-43a4-ab0b-34dab34d62ef.
Wrote to output file:  s3a://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-20

2020-08-01 10:03:33,049 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 8.0 (TID 10) in 4457247 ms on algo-2 (executor 1) (2/2)
2020-08-01 10:03:33,049 INFO cluster.YarnScheduler: Removed TaskSet 8.0, whose tasks have all completed, from pool 
2020-08-01 10:03:33,050 INFO scheduler.DAGScheduler: ResultStage 8 (save at NativeMethodAccessorImpl.java:0) finished in 4457.280 s
2020-08-01 10:03:33,050 INFO scheduler.DAGScheduler: Job 8 finished: save at NativeMethodAccessorImpl.java:0, took 4457.282827 s
2020-08-01 10:03:33,472 INFO datasources.FileFormatWriter: Write Job a5a4d70d-5818-426e-8dd3-651e7e3508a2 committed.
2020-08-01 10:03:33,472 INFO datasources.FileFormatWriter: Finished processing stats for write job a5a4d70d-5818-426e-8dd3-651e7e3508a2.
Wrote to output file:  s3a://sagemaker-us-east-1-322537213286/amazon-reviews-spark-processor-2020-08-01-06-14-52/output/bert-test
2020-08-01 10:03:33,653 INFO datasources.InMemoryFileIndex: It took 60 ms to list leaf files for 1 pa

2020-08-01 10:03:37
Finished Yarn configuration files setup.

Received end of job signal, exiting...
Finished Yarn configuration files setup.




<h2><span style="color:red">위 Processing Job이 완료되기 전까지 기다려 주시기 바랍니다.</span></h2>


# the Processed Output Dataset 확인

In [ ]:
!aws s3 ls --recursive $train_data_bert_output/

In [ ]:
!aws s3 ls --recursive $validation_data_bert_output/

In [ ]:
!aws s3 ls --recursive $test_data_bert_output/

In [ ]:
train_data = './data-tfrecord/bert-train'
validation_data = './data-tfrecord/bert-validation'
test_data = './data-tfrecord/bert-test'

!aws s3 cp $train_data_bert_output $train_data --recursive
!aws s3 cp $validation_data_bert_output $validation_data --recursive
!aws s3 cp $test_data_bert_output $test_data --recursive

In [ ]:
%store train_data_bert_output train_data

In [ ]:
%store validation_data_bert_output validation_data

In [ ]:
%store test_data_bert_output test_data